In [ ]:
import os
import sqlite3
import pandas as pd
from datetime import datetime
from io import StringIO

In [ ]:
base_dir = os.path.abspath(os.path.join(os.path.dirname(__file__), "..")) if '__file__' in globals() else os.path.abspath(os.path.join(os.getcwd(), ".."))
data_dir = os.path.join(base_dir, "data")

def process_metadata_file(file_path, table_name, conn):
    try:
        df = pd.read_csv(file_path, nrows=1)
        df.to_sql(table_name, conn, if_exists='replace', index=False)
        print(f"Metadata procesada en la tabla '{table_name}' desde el archivo {file_path}")
    except Exception as e:
        print(f"Error al procesar metadata en {file_path}: {e}")

def process_transaction_file(file_path, table_name, conn):
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            lines = f.readlines()
            
        header_index = None
        for i, line in enumerate(lines):
            if line.strip().startswith("Fecha de Transacción"):
                header_index = i
                break
        if header_index is None:
            print(f"No se encontró encabezado de transacciones en {file_path}")
            return
        
        csv_content = "".join(lines[header_index:])
        df = pd.read_csv(StringIO(csv_content))
        df.to_sql(table_name, conn, if_exists='append', index=False)
        print(f"Transacciones insertadas en la tabla '{table_name}' desde el archivo {file_path}")
    except Exception as e:
        print(f"Error al procesar transacciones en {file_path}: {e}")

def create_processed_files_table(conn):
    conn.execute("""
    CREATE TABLE IF NOT EXISTS processed_files (
        file_name TEXT PRIMARY KEY,
        folder_name TEXT,
        file_type TEXT,
        processed_date TEXT
    )
    """)
    conn.commit()

def is_file_processed(conn, file_name):
    cursor = conn.execute("SELECT 1 FROM processed_files WHERE file_name = ?", (file_name,))
    return cursor.fetchone() is not None

def is_metadata_file_updated(conn, file_name, file_path):
    cursor = conn.execute("SELECT processed_date FROM processed_files WHERE file_name = ? AND file_type = ?", (file_name, "metadata"))
    row = cursor.fetchone()
    if row is None:
        return True
    else:
        processed_date = datetime.fromisoformat(row[0])
        file_mod_time = datetime.fromtimestamp(os.path.getmtime(file_path))
        return file_mod_time > processed_date

def mark_file_as_processed(conn, file_name, folder_name, file_type):
    processed_date = datetime.now().isoformat()
    conn.execute("""
        INSERT OR REPLACE INTO processed_files (file_name, folder_name, file_type, processed_date) 
        VALUES (?, ?, ?, ?)
    """, (file_name, folder_name, file_type, processed_date))
    conn.commit()

def main():
    base_cleaned_path = os.path.join(data_dir, "cleanedData")
    db_path = os.path.join(data_dir, "dbs", "Transacciones_Bancarias_BAC.db")
    
    db_dir = os.path.dirname(db_path)
    if not os.path.exists(db_dir):
        os.makedirs(db_dir)
        
    conn = sqlite3.connect(db_path)
    create_processed_files_table(conn)

    for folder in os.listdir(base_cleaned_path):
        folder_path = os.path.join(base_cleaned_path, folder)
        if os.path.isdir(folder_path):
            metadata_table = folder + "_metadata"
            transactions_table = folder

            for file in os.listdir(folder_path):
                if file.lower().endswith(".csv"):
                    file_path = os.path.join(folder_path, file)
                    if "metadata" in file.lower():
                        if not is_metadata_file_updated(conn, file, file_path):
                            print(f"El archivo de metadata '{file}' no tiene datos nuevos. Se omite.")
                            continue
                        process_metadata_file(file_path, metadata_table, conn)
                        mark_file_as_processed(conn, file, folder, "metadata")
                    else:
                        if is_file_processed(conn, file):
                            print(f"El archivo '{file}' ya fue procesado. Se omite.")
                            continue
                        process_transaction_file(file_path, transactions_table, conn)
                        mark_file_as_processed(conn, file, folder, "transacciones")
    conn.close()

main()

## Consulta de tablas de Base de datos

In [ ]:
db_path = "../data/dbs/Transacciones_Bancarias_BAC.db"

output_folder = "../data/csv"

os.makedirs(output_folder, exist_ok=True)

conn = sqlite3.connect(db_path)
cursor = conn.cursor()

cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

for table in tables:
    table_name = table[0]
    print(f"Exportando tabla: {table_name}")

    df = pd.read_sql_query(f"SELECT * FROM {table_name}", conn)

    csv_file_path = os.path.join(output_folder, f"{table_name}.csv")

    df.to_csv(csv_file_path, index=False, encoding='utf-8-sig', sep=';')
    print(f"Tabla '{table_name}' exportada a: {csv_file_path}")

conn.close()
print("Exportación completada.")